In [ ]:
# https://github.com/tomasonjo/blogs/blob/master/neo4jdocs/neo4j_support_bot.ipynb
# %pip install langchain==0.0.142 openai==0.27.4 beautifulsoup4==4.12.2 chromadb==0.3.21

In [1]:
import os
import dotenv

# import requests
# from bs4 import BeautifulSoup
import pandas as pd

from langchain.document_loaders import (
    # GitLoader,
    # YoutubeLoader,
    # DataFrameLoader,
    SeleniumURLLoader,
)
from langchain.text_splitter import CharacterTextSplitter

# from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool

# loads .env file with your OPENAI_API_KEY
dotenv.load_dotenv()

True

### Embedding documents for the first time

In [ ]:
# urls = [
#     "https://kbw.com.sg/about-us/",
#     "https://kbw.com.sg/incorporation-locals/",
#     "https://kbw.com.sg/incorporation-foreigners/",
#     "https://kbw.com.sg/corp-secretary/",
#     "https://kbw.com.sg/resources-faqs/",
# ]

# loader = SeleniumURLLoader(urls=urls)
# data = loader.load()
# print(len(data))

In [ ]:
# # Define text chunk strategy
# splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separator=" ")
# # Split documents into chunks
# data_split = splitter.split_documents(data)
# print(len(data_split))

In [ ]:
# # Define embedding model
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# store = Chroma.from_documents(
#     data_split, embeddings, collection_name="kbw", persist_directory="db"
# )

# store.persist()
# store = None

### Chroma DB persisted

In [2]:
# Define embedding model
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
store = Chroma(
    collection_name="kbw", persist_directory="db", embedding_function=embeddings
)
store._collection

Using embedded DuckDB with persistence: data will be stored in: db


Collection(name=kwb)

In [ ]:
# # Delete collection
# store.delete_collection()
# store.persist()

In [ ]:
# # Lists all collection
# store._client.list_collections()

In [3]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    max_tokens=512,
)

In [4]:
sales_template = """
As a customer marketing bot, your goal is to provide accurate and helpful information about KBW Group, a top professional accounting firm in Singapore.
You should answer user inquiries based on the context provided and avoid making up answers.
If you don't know the answer, simply state that you don't know.
Remember to provide relevant information about how KBW can assist the user through its services, strengths and benefits.

{context}
=========
Question: {question}
"""

SALES_PROMPT = PromptTemplate(
    template=sales_template, input_variables=["context", "question"]
)

sales_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(),
    chain_type_kwargs={"prompt": SALES_PROMPT},
)

In [5]:
print(sales_qa.run("What does KBW provide?"))

KBW Group provides a range of professional accounting and business solutions, including company incorporation, corporate secretarial services, GST and tax advisory, accounting and payroll, external auditing, cloud system implementation, business valuation, and forensic accounting. They aim to provide reliable and innovative solutions to businesses of all sizes and industries, with a focus on client-centric solutions, integrity, innovation, and excellence.


In [6]:
support_template = """
As a customer support bot, you are here to assist with any issues a user might be facing with their company's accounting, advisory and/or audit.
This includes queries regarding the incorporation of a company.
Please provide as much detail as possible about the problem, how to solve it, and steps a user should take to fix it.
If the provided context doesn't provide enough information, you are allowed to use your knowledge and experience to offer the best possible assistance.

{context}
=========
Question: {question}
"""

SUPPORT_PROMPT = PromptTemplate(
    template=support_template, input_variables=["context", "question"]
)

support_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(),
    chain_type_kwargs={"prompt": SUPPORT_PROMPT},
)

In [7]:
print(support_qa.run("What do I need to incorporate a company in Singapore?"))

are the steps to incorporate a company in Singapore?
1. Choose a company name and check its availability on the ACRA website.
2. Prepare the necessary documents, including proof of identification and address for all individuals involved in the company.
3. Engage a corporate service provider to act as your company’s corporate secretary.
4. Register your company with ACRA and pay the necessary fees.
5. Obtain any necessary licenses or permits for your business activities.
6. Open a corporate bank account and deposit the required paid-up capital.
7. Obtain any necessary visas or work passes for foreign employees.
8. Start your business operations.


In [8]:
print(support_qa.run("How much does it cost?"))  # Doesn't include memory

How much does it cost to register a company in Singapore?
Registering a company in Singapore costs SGD 350, which includes the ACRA registration fee of S$300 and the name application fee of S$15. Additionally, you will need to pay for other services such as a registered office address and corporate secretary, which start from S$240 and S$60 respectively.


In [9]:
# the zero-shot-react-description agent will use the "description" string to select tools
tools = [
    Tool(
        name="sales",
        func=sales_qa.run,
        description="""
        Useful for when a user is interested in KBW information on accounting, advisory and/or auditing services. 
        A user is not asking for any solutions, but is only interested in general advice of the company, KBW.
        Input should be a fully formed question.
        """,
    ),
    Tool(
        name="support",
        func=support_qa.run,
        description="""
        Useful for when when a user asks to solve an issue or gets a query answered via
        specific instructions on how to accomplish a specified task. 
        Input should be a fully formed question.
        """,
    ),
]

In [10]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [11]:
agent.run(
    """What are the costs associated with incorporating a company in Singapore?"""
)



> Entering new AgentExecutor chain...
The user is asking for specific information on a process, so support may be the best option.
Action: support
Action Input: "What are the costs associated with incorporating a company in Singapore?"
Observation: are the costs associated with incorporating a company in Singapore?

The cost of incorporating a company in Singapore is SGD 350, which includes the ACRA registration fee of S$300 and the name application fee of S$15. Additionally, there may be other costs associated with incorporating a company, such as legal fees, accounting fees, and taxes. It is recommended to consult with a professional service provider to get a better understanding of the total costs involved in incorporating a company in Singapore.
Thought:The user may also be interested in the services KBW offers for incorporating a company, so sales may also be a good option.
Action: sales
Action Input: "What services does KBW offer for incorporating a company in Singapore?"
Obser

'The cost of incorporating a company in Singapore is SGD 350, which includes the ACRA registration fee of S$300 and the name application fee of S$15. KBW Group offers company incorporation services in Singapore, including corporate secretarial and corporate tax services, for a fee of S$1,500 per year and a refundable deposit of S$2,000. The deposit is to ensure that any outstanding fees or charges are settled before the company is officially incorporated, and will be refunded once all fees and charges have been settled. It is recommended to contact KBW Group directly for more information or to get started.'

In [ ]:
agent.run(
    """How can KBW assist my needs?"""
)  # Runs in a loop - needs some termination logic